Xử lý data gốc về data bỏ đầu và cuối, ngoài ra giúp đỡ lỗi về decord

In [1]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')


In [2]:
!kaggle datasets download -d bluebirdss/baole-dataset

100% 4.92G/4.93G [01:05<00:00, 123MB/s] 
100% 4.93G/4.93G [01:05<00:00, 80.8MB/s]


In [ ]:
!unzip baole-dataset.zip

Code xử lý data part 1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
import cv2

def process_video(input_video_path, output_video_path):
    # Khởi tạo đối tượng VideoCapture
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Lấy tổng số frames và FPS của video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    duration = total_frames / original_fps
    # Tính toán phần trăm và thời điểm bắt đầu, kết thúc để cắt
    if duration > 60:
        start_percent = 15
        end_percent = 85
    elif duration > 10:
        start_percent = 10
        end_percent = 90
    else:
        start_percent = 0
        end_percent = 100

    start_frame = int((start_percent / 100) * total_frames)
    end_frame = int((end_percent / 100) * total_frames)

    # Lấy kích thước frame của video
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Tạo đối tượng VideoWriter với FPS là desired_fps
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, original_fps, (frame_width, frame_height))

    # Đọc từng frame và ghi frame vào video mới nếu nằm trong khoảng đã tính và theo tỷ lệ FPS mới
    current_frame = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Chỉ xử lý frame nếu nằm trong khoảng thời gian đã định
        if start_frame <= current_frame <= end_frame:
            out.write(frame)

        current_frame += 1

    # Giải phóng tài nguyên
    cap.release()
    out.release()

    print("Video đã được xử lý và lưu tại: ", output_video_path)

In [ ]:
import os

data_dir = "/content/data_mae/test"
output_dir = "/content/output2/test"

for className in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, className)
    if os.path.isdir(class_dir):
        for video_file in os.listdir(class_dir):
            input_video_path = os.path.join(class_dir, video_file)
            filename_without_extension, _ = os.path.splitext(video_file)
            output_video_path = f"{output_dir}/{className}/{filename_without_extension}.mp4"
            print(filename_without_extension)
            os.makedirs(f"{output_dir}/{className}", exist_ok=True)
            process_video(input_video_path, output_video_path)


In [11]:
import os
import csv

def count_videos(dataset_path):
    """Count the number of videos in each class of the dataset and organize by class."""
    stats = {}
    for split in ['train', 'val']:
        split_path = os.path.join(dataset_path, split)
        if os.path.exists(split_path):
            for class_name in os.listdir(split_path):
                class_path = os.path.join(split_path, class_name)
                if os.path.isdir(class_path):
                    videos = [f for f in os.listdir(class_path) if f.endswith(('.mp4', '.MOV', '.mov', '.avi'))]
                    if class_name not in stats:
                        stats[class_name] = {'train': 0, 'val': 0}
                    stats[class_name][split] += len(videos)
    return stats

def save_stats_to_csv(stats, output_csv):
    """Save the dataset statistics to a CSV file with a structure that includes serial numbers and a total column."""
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['#', 'Class', 'Train', 'Val', 'Total'])
        serial_number = 1
        for class_name, splits in sorted(stats.items()):
            total_videos = splits['train'] + splits['val']
            writer.writerow([serial_number, class_name, splits['train'], splits['val'], total_videos])
            serial_number += 1

dataset_path = '/content/output2'
output_csv = 'dataset_statistics.csv'

# Count videos and save the statistics
stats = count_videos(dataset_path)
save_stats_to_csv(stats, output_csv)

print(f'Statistics saved to {output_csv}')

Statistics saved to dataset_statistics.csv


Chuyển video test (nếu có) vào val

In [ ]:
import os
import shutil

# Define the base paths for the test and val directories
base_test_path = '/content/output2/test'
base_val_path = '/content/output2/val'

# Get a list of class folders in the test directory
class_folders = os.listdir(base_test_path)

# Iterate over each class folder
for class_folder in class_folders:
    test_class_path = os.path.join(base_test_path, class_folder)
    val_class_path = os.path.join(base_val_path, class_folder)

    # Ensure the corresponding folder in val exists, if not, create it
    if not os.path.exists(val_class_path):
        os.makedirs(val_class_path)

    # Move each video file from the test class folder to the val class folder
    for file in os.listdir(test_class_path):
        # Check if the file is a video based on its extension
        if file.lower().endswith(('.mp4', '.mov', 'MOV')):
            # Define the source and destination paths for the file
            src_file_path = os.path.join(test_class_path, file)
            dest_file_path = os.path.join(val_class_path, file)

            # Move the file
            shutil.move(src_file_path, dest_file_path)
            print(f"Moved {file} from {test_class_path} to {val_class_path}")

print("Video files have been moved.")


In [16]:
cd ..

/content


In [ ]:
!zip -r '/content/drive/MyDrive/ai4life_2024/v2/v2_part1.zip' 'output2'

Download model

In [1]:
import requests

# Địa chỉ của API Flask
API_BASE_URL = 'http://103.188.243.119:5002'

def upload_file(filepath):
    """Tải lên một file lên server."""
    url = f'{API_BASE_URL}/upload'
    files = {'file': open(filepath, 'rb')}
    response = requests.post(url, files=files)
    return response.text

def download_file(filename):
    """Tải xuống một file từ server."""
    url = f'{API_BASE_URL}/download/{filename}'
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"File '{filename}' downloaded successfully.")
    else:
        print(f"Failed to download file: {response.text}")

# Đường dẫn đến file bạn muốn tải lên
# filepath_to_upload = '/content/yolov9/runs/train/exp/weights/best.pt'

# Tải lên file
# print(upload_file(filepath_to_upload))

# Tải xuống file
download_file('best_workout_3.pt')


File 'best_workout_3.pt' downloaded successfully.


In [ ]:
!git clone https://github.com/SkalskiP/yolov9.git
%cd yolov9
!pip install -r requirements.txt -q

In [ ]:
!gdown --id 1qo22TW0b83NpsXBHO4J12bGfPwDUpY5p

In [ ]:
!unzip v2_part1.zip

In [ ]:
!python detect.py --weights /content/best.pt --conf 0.1 --source /content/anhtest.png --device 0

1. Function: inference image

In [ ]:
import torch
from models.common import DetectMultiBackend
from utils.dataloaders import LoadImages
from utils.general import non_max_suppression, scale_boxes
from utils.torch_utils import select_device, smart_inference_mode

@smart_inference_mode()
def run_object_detection(weights='/content/best_workout_2.pt',
                         source='/content/anhtest.png',
                         imgsz=(640, 640),
                         conf_thres=0.25,
                         iou_thres=0.45,
                         device='0',
                         classes=None,
                         agnostic_nms=False):
    # Setup
    device = select_device(device)
    model = DetectMultiBackend(weights, device=device, dnn=False)

    # Dataloader
    dataset = LoadImages(source, img_size=imgsz, stride=model.stride, auto=model.pt)

    # List to store detections
    detections = []

    # Inference and processing
    for path, img, im0s, vid_cap, s in dataset:
        img = torch.from_numpy(img).to(device).float() / 255.0
        if len(img.shape) == 3:
            img = img[None]  # add batch dimension

        pred = model(img, augment=False, visualize=False)
        # Apply NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes=classes, agnostic=agnostic_nms)
        pred_converted = []

        for i, det in enumerate(pred):  # detections per image
            if len(det):
                det[:, :4] = scale_boxes(img.shape[2:], det[:, :4], im0s.shape).round()

                for *xyxy, conf, cls in reversed(det):
                    bbox = tuple(c.item() for c in xyxy)  # Convert bbox coordinates to tuple of floats
                    pred_converted.append((cls.item(), conf.item(), bbox))  # Convert class and confidence to floats

    return pred_converted

# Example usage
detections = run_object_detection()
print(detections)


YOLOv5 🚀 1e33dbb Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
gelan-c summary: 467 layers, 25411731 parameters, 0 gradients, 102.5 GFLOPs


[(0.0, 0.8926960825920105, (12.0, 97.0, 567.0, 362.0))]


Draw bounding box

In [ ]:
from PIL import Image, ImageDraw

# Load the original image
image_path = '/content/anhtest.png'
image = Image.open(image_path)
# image = image.resize((640, 640))
width_original, height_original = image.size

# Calculate the resize scale factors
scale_w = width_original / 640
scale_h = height_original / 640

detections_rescaled =detections
#  [
#     (cls, conf, [x_min * scale_w, y_min * scale_h, x_max * scale_w, y_max * scale_h])
#     for cls, conf, (x_min, y_min, x_max, y_max) in detections]

# Create a drawing context
draw = ImageDraw.Draw(image)

# Draw the rescaled bounding boxes on the original image
for det in detections_rescaled:
    cls, conf, bbox = det
    x_min, y_min, x_max, y_max = bbox
    draw.rectangle(((x_min, y_min), (x_max, y_max)), outline="red", width=2)
    label = f"Class: {cls}, Conf: {conf:.2f}"
    draw.text((x_min, y_min), label, fill="red")

# Display or save the image
# image.show()
# Or save to file
image.save('output_image_with_rescaled_boxes.png')


2. Function: Inference Video

In [8]:
import torch
from models.common import DetectMultiBackend
from utils.general import non_max_suppression, scale_boxes
from utils.torch_utils import select_device, smart_inference_mode
import cv2  # Import OpenCV
import numpy as np  # Ensure numpy is imported

# Correctly placed letterbox function
def letterbox(img, new_shape=(640, 640), color=(0, 0, 0)):
    """Resize image to a 32-pixel-multiple rectangle"""
    shape = img.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # width, height deltas
    dw, dh = np.mod(dw, 64), np.mod(dh, 64)  # divide padding into 2 sides
    dw /= 2
    dh /= 2
    if shape[::-1] != new_unpad:  # resize
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = round(dh - 0.1), round(dh + 0.1)
    left, right = round(dw - 0.1), round(dw + 0.1)
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return img

@smart_inference_mode()
def run_object_detection_video(weights='/content/best_workout_3.pt',
                               source='/content/data_mae/train/lateral raise/lateral_raise_6.MOV',
                               imgsz=(640, 640),
                               conf_thres=0.5,
                               iou_thres=0.5,
                               device='0',
                               classes=None,
                               agnostic_nms=False):
    # Setup
    device = select_device(device)
    model = DetectMultiBackend(weights, device=device, dnn=False)

    # Video capture
    vid_cap = cv2.VideoCapture(source)
    assert vid_cap.isOpened(), f'Failed to open video {source}'

    # List to store detections for each frame, including the frame number
    frame_detections = []

    frame_number = 0  # Initialize frame counter

    # Process each frame
    while True:
        ret, frame = vid_cap.read()
        if not ret:
            break

        frame_number += 1  # Increment frame counter

        # Convert BGR (OpenCV format) to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Resize and pad frame to model input size
        img = letterbox(frame, new_shape=imgsz)
        img = img.transpose((2, 0, 1))  # HWC to CHW
        img = np.ascontiguousarray(img)

        img = torch.from_numpy(img).to(device).float() / 255.0
        img = img[None]  # add batch dimension

        pred = model(img, augment=False, visualize=False)
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes=classes, agnostic=agnostic_nms)

        detections = []
        for i, det in enumerate(pred):  # detections per frame
            if len(det):
                det[:, :4] = scale_boxes(img.shape[2:], det[:, :4], frame.shape).round()

                for *xyxy, conf, cls in reversed(det):
                    bbox = tuple(c.item() for c in xyxy)  # Convert bbox coordinates to tuple of floats
                    # Now including frame_number in the detection information
                    detections.append((cls.item(), conf.item(), bbox, frame_number))

        if detections:  # If there are detections in the current frame
            frame_detections.append(detections)

    vid_cap.release()
    return frame_detections
# Ensure the main execution part of the script is correctly aligned and outside any function definition
video_detections = run_object_detection_video()
print(video_detections)


YOLOv5 🚀 1e33dbb Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
gelan-c summary: 467 layers, 25411731 parameters, 0 gradients, 102.5 GFLOPs


AssertionError: Failed to open video /content/data_mae/train/lateral raise/lateral_raise_6.MOV

In [ ]:
print(video_detections)

Draw bounding box for video

In [ ]:
import cv2

def draw_boxes_and_save_video(input_video_path, output_video_path, predictions):
    # Mở video gốc
    cap = cv2.VideoCapture(input_video_path)

    # Lấy thông số frame rate và định dạng size của video
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Định nghĩa codec và tạo đối tượng VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    frame_id = 1
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Vẽ bounding box cho từng frame dựa trên kết quả dự đoán
        for pred in predictions:
            pred = pred[0]
            # print(pred)
            if pred[3] == frame_id:  # Kiểm tra frame_id của prediction
                id_class, confidence, bbox, _ = pred
                x_min, y_min, x_max, y_max = bbox
                # Vẽ bounding box
                cv2.rectangle(frame, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (255, 0, 0), 2)
                # Vẽ text
                cv2.putText(frame, f'ID: {id_class}, Conf: {confidence:.2f}', (int(x_min), int(y_min) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

        # Viết frame đã chỉnh sửa vào file output
        out.write(frame)

        frame_id += 1

    # Giải phóng và đóng tất cả
    cap.release()
    out.release()

# Đường dẫn đến video gốc và video output
input_video_path = '/content/shoulder press_11.mp4'
output_video_path = 'output.mp4'

# Giả sử đây là kết quả dự đoán
# predictions = run_object_detection_video()

# Gọi hàm để vẽ bounding box và lưu video
draw_boxes_and_save_video(input_video_path, output_video_path, video_detections)

3. Function: Create video after detect and resize 224

In [7]:
import cv2

def crop_and_resize_frames(input_video_path, output_video_path, predictions):
    # Mở video gốc
    cap = cv2.VideoCapture(input_video_path)

    # Lấy thông số frame rate và định dạng size của video
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Định nghĩa codec và tạo đối tượng VideoWriter
    # Output size được set cố định là 224x224
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (224, 224))

    frame_id = 1
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        largest_bbox = None
        max_area = 0
        for pred in predictions:
            if pred[0][3] == frame_id:  # Kiểm tra frame_id của prediction
                _, _, bbox, _ = pred[0]
                x_min, y_min, x_max, y_max = bbox
                area = (x_max - x_min) * (y_max - y_min)
                if area > max_area:
                    max_area = area
                    largest_bbox = bbox
        if largest_bbox is not None:
            x_min, y_min, x_max, y_max = largest_bbox
            cropped_frame = frame[int(y_min):int(y_max), int(x_min):int(x_max)]
        else:
            # Nếu không có bbox, lấy toàn bộ khung hình
            cropped_frame = frame

        # Resize khung hình về 224x224
        resized_frame = cv2.resize(cropped_frame, (224, 224))

        # Viết frame đã chỉnh sửa vào file output
        out.write(resized_frame)

        frame_id += 1

    # Giải phóng và đóng tất cả
    cap.release()
    out.release()


# input_video_path = '/content/data_mae/train/lateral raise/lateral_raise_6.MOV'
# output_video_path = 'output.mp4'

# crop_and_resize_frames(input_video_path, output_video_path, video_detections)

Iteritor all video

In [ ]:
import os
data_dir = "/content/output2/train"
output_dir = "/content/output3/train"

for className in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, className)
        if os.path.isdir(class_dir):
            for video_file in os.listdir(class_dir):
                input_video_path = os.path.join(class_dir, video_file)
                output_video_path = f"{output_dir}/{className}/{video_file}"
                os.makedirs(f"{output_dir}/{className}", exist_ok=True)
                print(output_video_path)
                video_detections = run_object_detection_video(source = input_video_path)
                crop_and_resize_frames(input_video_path, output_video_path, video_detections)

YOLOv5 🚀 1e33dbb Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)



/content/output3/train/incline bench press/incline_bench_press_102.mp4


Fusing layers... 
gelan-c summary: 467 layers, 25411731 parameters, 0 gradients, 102.5 GFLOPs
YOLOv5 🚀 1e33dbb Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)



/content/output3/train/incline bench press/incline_bench_press_10.mp4


Fusing layers... 
gelan-c summary: 467 layers, 25411731 parameters, 0 gradients, 102.5 GFLOPs
YOLOv5 🚀 1e33dbb Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)



...

Fusing layers... 
gelan-c summary: 467 layers, 25411731 parameters, 0 gradients, 102.5 GFLOPs
YOLOv5 🚀 1e33dbb Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)



/content/output3/train/tricep pushdown/tricep_pushdown_22.mp4


Fusing layers... 
gelan-c summary: 467 layers, 25411731 parameters, 0 gradients, 102.5 GFLOPs
YOLOv5 🚀 1e33dbb Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)



/content/output3/train/tricep pushdown/tricep_pushdown_8.mp4


Fusing layers... 
gelan-c summary: 467 layers, 25411731 parameters, 0 gradients, 102.5 GFLOPs


In [ ]:
!zip -r '/content/drive/MyDrive/ai4life_2024/v2/v2_part2.zip' 'output3'